In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.display import Audio
import math
from multiprocessing import Process
import random
import sys
import time

from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective, SeriesMatchingObjective
from mlos.Optimizers.BayesianOptimizer import BayesianOptimizer
from mlos.Optimizers.BayesianOptimizerConfigStore import bayesian_optimizer_config_store
from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory
from mlos.Optimizers.OptimumDefinition import OptimumDefinition
from mlos.Optimizers.BayesianOptimizerConfigStore import bayesian_optimizer_config_store
from mlos.Optimizers.ExperimentDesigner.UtilityFunctionOptimizers.RandomNearIncumbentOptimizer import random_near_incumbent_optimizer_config_store
from mlos.Spaces import SimpleHypergrid, ContinuousDimension, DiscreteDimension, CategoricalDimension, Point

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.fft as fft
from scipy.stats import t

# Multi Series Convergence Demo

This notebook demonstrates how one would go about having multiple series converge at the same time. This notebook is meant as a continuation to **SeriesConvergenceDemo** and so this is mostly just demo code that proves that it works. I highly reccomend reading that first.

This is useful to have so that you can have multiple different objectives like (for example) CPU usage, RAM usage, Disk IO all of which have different underlying relationships to the initial parameters. (Another example would be things like replicating stock trading activity via measures of price, volatility, spread, etc over a day)


In [13]:
# The way the following functions are defined means that in order for the optimizer to succeed, it MUST use information from both functions as neither function has all required information
# to succeed at figuring out a,b, and c
def f1(x,a,b,c):
    return a + (b * x)

def f2(x,a,b,c):
    return b + (c * x)

series_domain_dimension = ContinuousDimension(name="time", min=0, max=10)
series_range_dimension_1 = ContinuousDimension(name="f1_output", min=0, max=1000)
series_range_dimension_2 = ContinuousDimension(name="f2_output", min=0, max=1000)

target_series_times = np.linspace(0,10,30).tolist()
target_series_values_1 = []
target_series_values_2 = []
#
# THE IDEAL END-PARAMETERS FOR THIS ARE : {a:1, b:2, c:3}
#
for t in target_series_times:
    target_series_values_1.append(f1(t,1,2,0))
    target_series_values_2.append(f2(t,0,2,3))
    
target_series_1_df = pd.DataFrame({
    "time":target_series_times,
    "f1_output": target_series_values_1
})

target_series_2_df = pd.DataFrame({
    "time":target_series_times,
    "f2_output": target_series_values_2
})


## Optimizer Definition

In [14]:
input_space = SimpleHypergrid(
    name="input",
    dimensions=[
        ContinuousDimension(name="a", min=0, max=10),
        DiscreteDimension(name="b", min=0, max=10),
        ContinuousDimension(name="c", min=0, max=10)
    ]
)

output_space = SimpleHypergrid(
    name="output",
    dimensions=[
        ContinuousDimension(name="f1_fit", min=0, max=10**10),
        ContinuousDimension(name="f2_fit", min=0, max=10**10)
    ]
)

optimization_problem = OptimizationProblem(
        parameter_space=input_space,
        objective_space=output_space,
        objectives=[
            SeriesMatchingObjective(
                name="f1_fit", 
                target_series_df=target_series_1_df, 
                series_domain_dimension=series_domain_dimension,
                series_codomain_dimension=series_range_dimension_1,
                series_difference_metric="sum_of_squared_errors"
            ),
            SeriesMatchingObjective(
                name="f2_fit", 
                target_series_df=target_series_2_df, 
                series_domain_dimension=series_domain_dimension,
                series_codomain_dimension=series_range_dimension_2,
                series_difference_metric="sum_of_squared_errors"
            )
        ]
    )

In [15]:
optimizer_config = bayesian_optimizer_config_store.default

optimizer_config.min_samples_required_for_guided_design_of_experiments = 20
optimizer_config.homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.n_new_samples_before_refit = 3
optimizer_config.homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator = 0.8
optimizer_config.homogeneous_random_forest_regression_model_config.n_estimators = 25
optimizer_config.experiment_designer_config.numeric_optimizer_implementation = "RandomNearIncumbentOptimizer"
optimizer_config.experiment_designer_config.random_near_incumbent_optimizer_config = random_near_incumbent_optimizer_config_store.default#["20_incumbents_50_neighbors"]
optimizer_config.experiment_designer_config.fraction_random_suggestions = 0.6


optimizer_factory = BayesianOptimizerFactory()
optimizer = optimizer_factory.create_local_optimizer(
    optimization_problem=optimization_problem,
    optimizer_config=optimizer_config
)

08/13/2021 10:38:24 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizerFactory.py:  43 -    create_local_optimizer() ] Creating a bayesian optimizer with config: {
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 20,
  "homogeneous_random_forest_regression_model_config.n_estimators": 25,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 1,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.8,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.criterion": "mse",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.splitter": "best",
  "homogeneous_random_forest_regression_m

08/13/2021 10:38:24 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecisionTreeRegressionModel with the input_space:   Name: estimator_10_input_space
  Dimensions:
    contains_context: {False}
    input___b: {0, 1, ... , 10}
    series_context_space___time: [0.00, 10.00]
    input___a: [0.00, 10.00]
    input___c: [0.00, 10.00]
08/13/2021 10:38:24 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecisionTreeRegressionModel with the input_space:   Name: estimator_11_input_space
  Dimensions:
    contains_context: {False}
    input___b: {0, 1, ... , 10}
    input___c: [0.00, 10.00]
    input___a: [0.00, 10.00]
    series_context_space___time: [0.00, 10.00]
08/13/2021 10:38:24 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecisionTreeRegression

08/13/2021 10:38:24 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecisionTreeRegressionModel with the input_space:   Name: estimator_4_input_space
  Dimensions:
    input___a: [0.00, 10.00]
    input___c: [0.00, 10.00]
    contains_context: {False}
    series_context_space___time: [0.00, 10.00]
    input___b: {0, 1, ... , 10}
08/13/2021 10:38:24 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecisionTreeRegressionModel with the input_space:   Name: estimator_5_input_space
  Dimensions:
    contains_context: {False}
    input___b: {0, 1, ... , 10}
    input___a: [0.00, 10.00]
    input___c: [0.00, 10.00]
    series_context_space___time: [0.00, 10.00]
08/13/2021 10:38:24 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecisionTreeRegressionMo

08/13/2021 10:38:24 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecisionTreeRegressionModel with the input_space:   Name: estimator_24_input_space
  Dimensions:
    contains_context: {False}
    input___c: [0.00, 10.00]
    series_context_space___time: [0.00, 10.00]
    input___a: [0.00, 10.00]
    input___b: {0, 1, ... , 10}


ZACK CREATING SERIES AWARE MULTI OBJECTIVE BLAH
ZACK KAPPA
ZACK KAPPA


## Optimization Loop

**Warning**: this can take a very long time (>1hour). The RandomNearIncumbentOptimizer can be quite slow.

In [16]:
for current_iteration in range(200):
    print(f"Current iteration: {current_iteration}")
    
    suggested_value = optimizer.suggest()
    input_values_df = suggested_value.to_dataframe()
    
    f1_values = []
    f2_values = []
    for time in target_series_times:
        f1_at_time = f1(
            time,
            suggested_value["a"],
            suggested_value["b"],
            suggested_value["c"]
        )
        f2_at_time = f2(
            time,
            suggested_value["a"],
            suggested_value["b"],
            suggested_value["c"]
        )
        f1_values.append(f1_at_time)
        f2_values.append(f2_at_time)
        
    context_values_df = pd.DataFrame({
        series_domain_dimension.name: [target_series_times]
    })
    
    target_values_df = pd.DataFrame({
        series_range_dimension_1.name: [f1_values],
        series_range_dimension_2.name: [f2_values]
    })
    
    # REGISTER
    #
    optimizer.register(input_values_df, target_values_df, context_values_df)
    

08/13/2021 10:38:28 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 186 -                   suggest() ] Producing random suggestion: {
  "a": 0.9543482951633109,
  "b": 0,
  "c": 2.753015977874309
}
08/13/2021 10:38:28 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 163 -                  register() ] Registering 1 parameters and 1 objectives.
08/13/2021 10:38:28 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 186 -                   suggest() ] Producing random suggestion: {
  "a": 9.516021752384411,
  "b": 2,
  "c": 0.7342119536612046
}
08/13/2021 10:38:28 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 163 -                  register() ] Registering 1 parameters and 1 objectives.
08/13/2021 10:38:28 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 186 -                   suggest() ] Producing random suggestion: {
  "a": 3.028740907276225,
  "b": 9,
  "c": 0.08014754168781613
}
08/13/2021 10:38:28 -   B

Current iteration: 0
Current iteration: 1
Current iteration: 2
Current iteration: 3
Current iteration: 4
Current iteration: 5
Current iteration: 6
Current iteration: 7
Current iteration: 8
Current iteration: 9
Current iteration: 10


08/13/2021 10:38:28 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 163 -                  register() ] Registering 1 parameters and 1 objectives.
08/13/2021 10:38:28 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 186 -                   suggest() ] Producing random suggestion: {
  "a": 1.2848452135292632,
  "b": 4,
  "c": 3.9835841548683093
}
08/13/2021 10:38:28 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 163 -                  register() ] Registering 1 parameters and 1 objectives.
08/13/2021 10:38:28 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 186 -                   suggest() ] Producing random suggestion: {
  "a": 9.781967970892607,
  "b": 10,
  "c": 6.693245926459507
}
08/13/2021 10:38:28 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 163 -                  register() ] Registering 1 parameters and 1 objectives.
08/13/2021 10:38:28 -   BayesianOptimizerFactory -    INFO - [ExperimentDesign

Current iteration: 11
Current iteration: 12
Current iteration: 13
Current iteration: 14
Current iteration: 15
Current iteration: 16
Current iteration: 17
Current iteration: 18
Current iteration: 19


08/13/2021 10:38:30 -   BayesianOptimizerFactory -    INFO - [ExperimentDesigner.py: 186 -                   suggest() ] Producing random suggestion: {
  "a": 8.309367685843412,
  "b": 4,
  "c": 6.953098741281608
}
08/13/2021 10:38:30 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizer.py: 163 -                  register() ] Registering 1 parameters and 1 objectives.


...

Current iteration: 104


08/13/2021 11:02:22 -   BayesianOptimizerFactory -    INFO - [RandomNearIncumbentOptimizer.py: 165 -                   suggest() ] Suggesting config for context: None
08/13/2021 11:02:22 -   BayesianOptimizerFactory -    INFO - [RandomNearIncumbentOptimizer.py: 316 - _prepare_initial_params_df() ] Preparing initial params
08/13/2021 11:02:22 -   BayesianOptimizerFactory -    INFO - [RandomNearIncumbentOptimizer.py: 342 - _prepare_initial_params_df() ] Using 5 of 8 pareto points as starting configs.
08/13/2021 11:02:22 -   BayesianOptimizerFactory -    INFO - [RandomNearIncumbentOptimizer.py: 355 - _prepare_initial_params_df() ] Using 3 out of 440 cached good configs as starting configs
08/13/2021 11:02:22 -   BayesianOptimizerFactory -    INFO - [RandomNearIncumbentOptimizer.py: 366 - _prepare_initial_params_df() ] Using 2 random points as starting configs.


Current iteration: 105


08/13/2021 11:02:23 -   BayesianOptimizerFactory -    INFO - [RandomNearIncumbentOptimizer.py: 419 - _prepare_random_neighbors() ] Num active incumbents: 10/10, num neighbors per incumbent: 20
08/13/2021 11:02:23 -   BayesianOptimizerFactory -    INFO - [RandomNearIncumbentOptimizer.py: 472 - _prepare_random_neighbors() ] Started with 200. Adapter filtered them down to 81. Parameter space filtered them down to 81
08/13/2021 11:02:24 -   BayesianOptimizerFactory -    INFO - [RandomNearIncumbentOptimizer.py: 240 -            _run_iteration() ] [1]Computed utility for 81 random neighbors.
08/13/2021 11:02:24 -   BayesianOptimizerFactory -    INFO - [RandomNearIncumbentOptimizer.py: 249 -            _run_iteration() ] 11 have positive utility gain.
08/13/2021 11:02:24 -   BayesianOptimizerFactory -    INFO - [RandomNearIncumbentOptimizer.py: 256 -            _run_iteration() ] 4 neighbors improved upon their respective incumbents.
08/13/2021 11:02:24 -   BayesianOptimizerFactory -    INFO 

KeyboardInterrupt: 

## Results

In [19]:
optimizer.pareto_frontier.pareto_df

,f1_fit,f2_fit
38,0.124390,44339.680103
55,0.151498,55.794615
53,0.181354,15.813251
59,0.181354,15.813251
60,0.181354,15.813251
61,0.181354,15.813251
62,0.181354,15.813251
65,0.181354,15.813251


In [20]:
optimizer.pareto_frontier.params_for_pareto_df

,a,b,c
38,0.954651,2,9.596315
55,0.954570,2,3.303173
53,0.956974,2,3.100619
59,0.956974,2,3.100619
60,0.956974,2,3.100619
61,0.956974,2,3.100619
62,0.956974,2,3.100619
65,0.956974,2,3.100619


In [23]:
# Obviously from the previous table, it has found the 1, 2, 3 location as the best. The optimizer just doesn't pick the right optimum

optimizer.optimum()

({
   "a": 0.9546514550035934,
   "b": 2,
   "c": 9.596314563738733
 },
 {
   "f1_fit": 0.1243902081875011,
   "f2_fit": 44339.68010309439,
   "f1_output": [
     0.9546514550035934,
     1.6443066274173865,
     2.3339617998311795,
     3.023616972244973,
     3.7132721446587658,
     4.402927317072559,
     5.092582489486352,
     5.782237661900145,
     6.471892834313938,
     7.161548006727732,
     7.851203179141525,
     8.540858351555318,
     9.230513523969112,
     9.920168696382905,
     10.609823868796697,
     11.29947904121049,
     11.989134213624284,
     12.678789386038078,
     13.368444558451872,
     14.058099730865663,
     14.747754903279457,
     15.43741007569325,
     16.12706524810704,
     16.816720420520834,
     17.506375592934628,
     18.19603076534842,
     18.885685937762215,
     19.57534111017601,
     20.2649962825898,
     20.954651455003592
   ],
   "f2_output": [
     2.0,
     5.309073987496115,
     8.61814797499223,
     11.927221962488346,
    